# Data Scraping en utilisant la librairie Snscrape
> #### [Snscrape](https://github.com/JustAnotherArchivist/snscrape) est un scraper pour les services de réseaux sociaux. Il récupère des éléments tels que les profils d'utilisateurs, les hashtags ou les recherches et renvoie les éléments découverts, par ex. les postes concernés.



### Installation du dernier version de la librairie Snscrape

In [ ]:
! pip install git+https://github.com/JustAnotherArchivist/snscrape.git

### Importation des librairies

In [2]:
import os
import shutil
import pandas as pd
import json
import zipfile

### Snscraping

##### Définition des paramètres de Scraper des données

In [3]:
# Requête (recherche de texte)
  # Chercher les tweets du dernier mois
tweet_count = 100000
text_query = "covid lang:fr"
since_date = "2021-12-06"
until_date = "2022-01-08"
output_file = "covid-tweets.json"

##### Utilisation de la bibliothèque OS pour appeler des commandes CLI en Python

In [ ]:
os.system(f'snscrape --jsonl --max-results { tweet_count } twitter-search "{ text_query } since:{ since_date }  until:{ until_date }" > { output_file }')

##### Lecture du fichier json généré à partir de la commande CLI ci-dessus et crée une dataframe pandas

In [5]:
tweets_file = output_file
cols = ['date', 'content']
data = []

with open(tweets_file, encoding='utf-16') as file: # Enlever le encoding s'il y'a une erreur.
    for line in file.readlines():
      doc = json.loads(line)
      lst = [doc['date'], doc['content']]
      data.append(lst)

tweets_df = pd.DataFrame(data = data, columns = cols)

##### Echantillon de données

In [6]:
tweets_df.head()

,date,content
0,2022-01-07T08:56:07+00:00,M. le 1er Ministre @JeanCASTEX \nM. le Ministr...
1,2022-01-07T08:55:59+00:00,Qui est l’ancêtre du père Noël ?! 🎅🏾😂 @Bengous...
2,2022-01-07T08:54:53+00:00,"Non, le vaccin contre la #Covid19 n’est plus e..."
3,2022-01-07T08:54:53+00:00,@FEvan96488906 @herve66pinson @christinedorin ...
4,2022-01-07T08:54:49+00:00,Football. Covid19. Le Syli enregistre de nouve...


### Création du corpus

##### Création du répertoire

In [7]:
corpusdir = 'tweets/'
if not os.path.isdir(corpusdir):
    os.mkdir(corpusdir)

##### Exportation des tweets dans un fichier de nom 'tweet_{id}_{date}.txt'

In [ ]:
for id, row in tweets_df.iterrows():
  filename = f'tweet_{ id }_{ pd.to_datetime(row["date"]).date() }.txt'
  with open(corpusdir + filename, 'w', encoding="utf-8") as file:
    file.write(row['content'])
  with zipfile.ZipFile("corpus_tweets.zip", "a", compression = zipfile.ZIP_DEFLATED) as zf:
    zf.write(corpusdir + filename)

##### Suppréssion des fichiers inutiles

In [9]:
if os.path.isdir(corpusdir):
  shutil.rmtree(corpusdir)

os.remove(tweets_file)